In [1]:
import pandas as pd
import numpy as np
import os
import ollama
import re
_dir_ = '../../../data/2014_vast/MC3/'
_embeddings_filename_ = '../../../data/2014_vast/MC3/embeddings.parquet'
if os.path.exists(_embeddings_filename_) == False:
    _dfs_ = []
    for _file_ in os.listdir(_dir_):
        if _file_.endswith('csv'):
            _df_ = pd.read_csv(_dir_+_file_, encoding='iso-8859-1')
            _dfs_.append(_df_)
    df              = pd.concat(_dfs_)
    df['hash_tags'] = df['message'].apply(lambda s: re.findall(r'#\w+',   s))
    df['reply_to']  = df['message'].apply(lambda s: re.findall(r'@(\w+)', s))
    embeddings = []
    for i in range(len(df)):
        _df_      = df.iloc[i]
        _message_ = df.iloc[i]['message']
        _response_ = ollama.embeddings(model='nomic-embed-text', prompt=_message_)
        embeddings.append(_response_['embedding'])
    df['embedding'] = embeddings
    df.to_parquet(_embeddings_filename_)
else:
    df = pd.read_parquet(_embeddings_filename_)